## RECONCILE 2 DATASETS with 4 columns with categorical data as INDEX


In [1]:
import pandas as pd

In [2]:
df1=pd.read_excel("D:\PERSO\DATA SCIENCE\Test_GWP_to_NWP_NO_INDEX.xlsx", sheet_name="DS1")

# DF1 - Clean and rename columns, reorder columns

df1["Entity"] = '000'+df1["SYND"].astype(str)
df1["NEW_YoA"] = df1["YoA"].astype(str)
df1 = df1.drop(["SYND", "YoA"], axis=1)
df1.rename(columns={"NEW_YoA":"YoA"}, inplace=True)
df1 = df1[["Entity","YoA","Code","CCY", "GWP","Comms" ]]
df1.head(5)

,Entity,YoA,Code,CCY,GWP,Comms
0,0001,2000,TARC1,EUR,12000.0,3000.0
1,0001,2001,TarcXY,EUR,13200.0,3300.0
2,0001,2002,ARC3T,EUR,14520.0,3630.0
3,0001,2003,RC4TA,EUR,15972.0,3993.0
4,0001,2004,RAC5T,EUR,17569.2,4392.3


In [4]:
df2=pd.read_excel("D:\PERSO\DATA SCIENCE\Test_GWP_to_NWP_NO_INDEX.xlsx", sheet_name="DS2")

# DF1 - Clean and rename columns, reorder columns

df2["Entity"] = '000'+df2["SYND"].astype(str)
df2["NEW_YoA"] = df2["YoA"].astype(str)
df2 = df2.drop(["SYND", "YoA"], axis=1)
df2.rename(columns={"NEW_YoA":"YoA"}, inplace=True)
df2 = df2[["Entity","YoA","Code","CCY", "GWP","Comms" ]]
df2.head(5)

,Entity,YoA,Code,CCY,GWP,Comms
0,0001,2000,TARC1,EUR,12000.0,3000.0
1,0001,2001,TarcXY,EUR,13200.0,3300.0
2,0001,2002,ARC3T,EUR,14520.0,3630.0
3,0001,2003,RC4TA,EUR,15972.0,3993.0
4,0001,2004,RAC5T,EUR,17569.2,4392.3


## USING DATAFRAMES ONLY
*****************

In [5]:
df1["NWP"] = df1["GWP"] - df1["Comms"]
df2["NWP"] = df2["GWP"] - df2["Comms"]
df1["Source"]="Source1"
df2["Source"]="Source2"

In [6]:
df1 = df1[["Source","Entity","YoA","Code","CCY", "GWP","Comms", "NWP"]]
df2 = df2[["Source","Entity","YoA","Code","CCY", "GWP","Comms", "NWP"]]

In [7]:
df_total= pd.concat([df1, df2], axis=0)
df_total.head(3)

,Source,Entity,YoA,Code,CCY,GWP,Comms,NWP
0,Source1,0001,2000,TARC1,EUR,12000.0,3000.0,9000.0
1,Source1,0001,2001,TarcXY,EUR,13200.0,3300.0,9900.0
2,Source1,0001,2002,ARC3T,EUR,14520.0,3630.0,10890.0


In [8]:
# ANALYSIS FOR GWP ONLY

df1_i = df1.set_index(["Entity","YoA","Code","CCY"])
df2_i = df2.set_index(["Entity","YoA","Code","CCY"])
df_diff = df1_i.join(df2_i,how='outer',rsuffix='_').fillna(0)

# df_diff_GWP = (df_diff['GWP'] - df_diff['GWP_'])
# df_diff_GWP.head(5)

In [9]:
df_diff.head(3)

Source      GWP   Comms      NWP  Source_     GWP_  \
Entity YoA  Code   CCY                                                        
0001   2000 TARC1  EUR  Source1  12000.0  3000.0   9000.0  Source2  12000.0   
       2001 TarcXY EUR  Source1  13200.0  3300.0   9900.0  Source2  13200.0   
       2002 ARC3T  EUR  Source1  14520.0  3630.0  10890.0  Source2  14520.0   

                        Comms_     NWP_  
Entity YoA  Code   CCY                   
0001   2000 TARC1  EUR  3000.0   9000.0  
       2001 TarcXY EUR  3300.0   9900.0  
       2002 ARC3T  EUR  3630.0  10890.0

In [10]:
df_diff.rename(columns={"Source_":"Source2","GWP_":"GWP2","Comms_":"Comms2" ,"NWP_":"NWP2"}, inplace=True)

In [11]:
df_diff["GWP_diff"]=df_diff["GWP"]-df_diff["GWP2"]
df_diff["Comms_diff"]=df_diff["Comms"]-df_diff["Comms2"]
df_diff["NWP_diff"]=df_diff["NWP"]-df_diff["NWP2"]

In [12]:
df_diff.head(3)

Source      GWP   Comms      NWP  Source2     GWP2  \
Entity YoA  Code   CCY                                                        
0001   2000 TARC1  EUR  Source1  12000.0  3000.0   9000.0  Source2  12000.0   
       2001 TarcXY EUR  Source1  13200.0  3300.0   9900.0  Source2  13200.0   
       2002 ARC3T  EUR  Source1  14520.0  3630.0  10890.0  Source2  14520.0   

                        Comms2     NWP2  GWP_diff  Comms_diff  NWP_diff  
Entity YoA  Code   CCY                                                   
0001   2000 TARC1  EUR  3000.0   9000.0       0.0         0.0       0.0  
       2001 TarcXY EUR  3300.0   9900.0       0.0         0.0       0.0  
       2002 ARC3T  EUR  3630.0  10890.0       0.0         0.0       0.0

In [13]:
reconciling_items = df_diff["GWP_diff"]!=0
reconciling_items_df= df_diff[reconciling_items]

In [14]:
# Find TODAY's date and print in multitab spreadsheet

p = pd.Timestamp.now().to_period('D')
today = p.to_timestamp().strftime("%Y-%m-%d")

Filepath = "D:\PERSO\DATA SCIENCE\MyRecs_"+today+'.xlsx'

with pd.ExcelWriter(Filepath,) as writer:
    df_diff.to_excel(writer,sheet_name="My_sheet_no_1", index=False)
    reconciling_items_df.to_excel(writer,sheet_name="My_sheet_no_2", index=False)
    print("File created!")

File created!


******************************

In [15]:
df_diff.to_excel("D:/PERSO/Final_Rec.xlsx")

------------------------

In [ ]:
# connect to SSAS cube

import pyodbc
import sqlalchemy
import pandas as pd
import re

import clr 
# (pythonnet)

clr.AddReference(r"C:Users\MySelf\my_path\Microsoft.AnalysisServices.AdomdClient.dll")
clr.AddReference("System.Data")

from Microsoft.AnalsysisServices.AdomdClient import AdomdConnection, AdomdDataAdapter
from System.Data import DataSet

In [ ]:
from mdx_to_pd import mdx_retriever

connection = AdomdConnection("Data Dource=SERVER01.companynet.corp.local;Catalog=InterestingCube;Cube=InformFinanceCube")

My_query = """
MDX query
"""

DF_what = mdx_retriever(My_query, connection)